In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import time
from datetime import datetime
from account_credentials import LOGIN, PASSWORD, SERVER
import plotly.express as px

In [ ]:
mt5.initialize()
login = LOGIN
password = PASSWORD
server = SERVER
mt5.login(login, password, server)

In [ ]:
# getting account info
account_info = mt5.account_info()
print(account_info)

# getting specific account data
login_number = account_info.login
balance = account_info.balance
equity = account_info.equity

print()
print('login: ', login_number)
print('balance: ', balance)
print('equity: ', equity)

In [ ]:
# get number of symbols with symbols_total()
num_symbols = mt5.symbols_total()
num_symbols

In [ ]:
# get all symbols and their specifications
symbols = mt5.symbols_get()

all_symbols = []
for symbol in symbols:
    all_symbols.append(symbol.name)

search_symbol = 'XAUUSD'
if search_symbol in all_symbols:
    print('{} is available'.format(search_symbol))
else:
    print('{} is not available',format(search_symbol))

symbol = search_symbol

In [ ]:
# to view the specific symbols details
symbol_info = mt5.symbol_info_tick(symbol)._asdict()
symbol_info

In [ ]:
symbol = 'XAUUSD'
timeframe = mt5.TIMEFRAME_H1
volume = 0.5
strategy_name = 'ma_trendfollowing'

In [ ]:
# get candle stick data
candle = pd.DataFrame(mt5.copy_rates_range(symbol,
                                           timeframe,
                                           datetime(2021,1,1),
                                           datetime.now()
                                           ))
candle['time'] = pd.to_datetime(candle['time'], unit='s')

fig = px.line(candle, x = candle['time'], y = candle['close'])
fig.show()

candle.head()

In [ ]:
# get ticks  data
tick_data = pd.DataFrame(mt5.copy_ticks_range(symbol,
                                           datetime(2023,1,1),
                                           datetime.now(),
                                           mt5.COPY_TICKS_ALL
                                           ))
tick_data['time'] = pd.to_datetime(tick_data['time'], unit='s')

# fig = px.line(tick_data, x=tick_data['time'], y = tick_data['bid'])
# fig.show()

print(len(tick_data))
tick_data.head()

In [ ]:
# get number of total open order, including pending order
num_orders = mt5.orders_total()
num_orders

In [ ]:
# list of orders in details
orders = mt5.orders_get()
orders

In [ ]:
# total number of positions
num_positions = mt5.positions_total()
num_positions

In [ ]:
# position detailf
positions = mt5.positions_get()
positions

In [ ]:
# Send order to the market
# documentation: https://www.mql5.com/en/docs/python_metatrader5/mt5ordersend_py

request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": 2.0, #float,
    "type": mt5.ORDER_TYPE_BUY,
    "price": mt5.symbol_info_tick(symbol).ask,
    "sl": 0.0, # float
    "tp": 0.0, # float
    "deviation": 0, # integer
    "magic": 123488, # integer
    "comment": "sent by python",
    "type_time": mt5.ORDER_TIME_GTC, # sometime this depend on broker
    "type_filling": mt5.ORDER_FILLING_IOC, # sometime this depend on broker
}
order = mt5.order_send(request)
print(order)

In [ ]:
# Close order to the market
# documentation: https://www.mql5.com/en/docs/python_metatrader5/mt5ordersend_py

request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": 2.0, #float,
    "type": mt5.ORDER_TYPE_SELL,
    "position": 111111, # select the position you want to close
    "price": mt5.symbol_info_tick(symbol).ask,
    "sl": 0.0, # float
    "tp": 0.0, # float
    "deviation": 0, # integer
    "magic": 123488, # integer
    "comment": "sent by python",
    "type_time": mt5.ORDER_TIME_GTC, # sometime this depend on broker
    "type_filling": mt5.ORDER_FILLING_IOC, # sometime this depend on broker
}
order = mt5.order_send(request)
print(order)

# Start implementing

#### Testing API

In [ ]:
import requests

url = "http://127.0.0.1:5000/predict_api" # replace with your API endpoint URL
data = {
	"data": {
		"open": 1910,
		"high": 1920,
		"low": 1900,
		"close": 1913
	}
} # replace with your JSON data

response = requests.post(url, json=data)

if response.status_code == 200:
    print("POST request successful!")
    print(response.json())
else:
    print("POST request failed!")
    print(response.status_code)

# Implementing

In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import time
from datetime import datetime
from account_credentials import LOGIN, PASSWORD, SERVER
import plotly.express as px

In [ ]:
symbol = 'XAUUSD'
timeframe = mt5.TIMEFRAME_H1
volume = 0.01
strategy_name = 'ML_lsvm'

In [ ]:
def check_allowed_trading_hours():
    tick = mt5.symbol_info_tick(symbol)
    # check the last price value to determine if the market is closed or available
    if tick.last == 0:
        #market close
        return False
    else:
        #market open
        return True

In [ ]:
def market_order(symbol, volume, order_type, deviation=0, magic=123999):

    order_type_dict = {
        'buy': mt5.ORDER_TYPE_BUY,
        'sell': mt5.ORDER_TYPE_SELL
    }

    price_dict = {
        'buy': mt5.symbol_info_tick(symbol).ask,
        'sell': mt5.symbol_info_tick(symbol).bid
    }

    if order_type == 'buy':
        sl = mt5.symbol_info_tick(symbol).ask - 3
        tp = mt5.symbol_info_tick(symbol).ask + 3.3
    
    if order_type == 'sell':
        sl = mt5.symbol_info_tick(symbol).bid + 3
        tp = mt5.symbol_info_tick(symbol).bid - 3.3

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,  # FLOAT
        "type": order_type_dict[order_type],
        "price": price_dict[order_type],
        "sl": sl,  # FLOAT
        "tp": tp,  # FLOAT
        "deviation": deviation,  # INTERGER
        "magic": magic,  # INTERGER
        "comment": strategy_name,
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    order_result = mt5.order_send(request)
    return(order_result)

In [ ]:
def close_position(position, deviation=0, magic=123999):

    order_type_dict = {
        0: mt5.ORDER_TYPE_SELL,
        1: mt5.ORDER_TYPE_BUY
    }

    price_dict = {
        0: mt5.symbol_info_tick(symbol).bid,
        1: mt5.symbol_info_tick(symbol).ask
    }

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "position": position['ticket'],  # select the position you want to close
        "symbol": symbol,
        "volume": volume,  # FLOAT
        "type": order_type_dict[position['type']],
        "price": price_dict[position['type']],
        "deviation": deviation,  # INTERGER
        "magic": magic,  # INTERGER
        "comment": strategy_name,
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    order_result = mt5.order_send(request)
    return(order_result)

def close_positions(order_type):
    order_type_dict = {
        'buy': 0,
        'sell': 1
    }

    if mt5.positions_total() > 0:
        positions = mt5.positions_get()

        positions_df = pd.DataFrame(positions, columns=positions[0]._asdict().keys())

        if order_type != 'all':
            positions_df = positions_df[(positions_df['type'] == order_type_dict[order_type])]

        for i, position in positions_df.iterrows():
            order_result = close_position(position)

            print('order_result: ', order_result)

In [ ]:
if __name__ == '__main__':
    is_initialized = mt5.initialize()
    print('initialize: ', is_initialized)

    is_logged_in = mt5.login(LOGIN, PASSWORD, SERVER)
    print('logged in: ', is_logged_in)
    print('\n')

1

In [52]:
#### RUN ONCE TO CREATE A RECORD.CSV FILE
time_records = pd.read_csv('time_records.csv')
if len(time_records) > 0:
    print('Your already have a time_records file: CONTINUE')
else:
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, 1)
    price_data = rates[0]
    open_price = price_data[1]
    high_price = price_data[2]
    low_price = price_data[3]
    close_price = price_data[4]
    time_trade = datetime.fromtimestamp(price_data[0])

    time_records = [time_trade]
    records_df = pd.DataFrame({'time_records': time_records})
    records_df.to_csv('time_records.csv', index=False)
    print('Created a time_records file')

Your already have a time_records file: CONTINUE


In [ ]:
from datetime import datetime

time_records = pd.read_csv('time_records.csv')
while True:
    account_info = mt5.account_info()
    print(datetime.now(),
            '| Login: ', account_info.login,
            '| Balance: ', account_info.balance,
            '| Equity: ' , account_info.equity)
    num_positions = mt5.positions_total()
    print('number of positions: ',num_positions)

    if check_allowed_trading_hours() == False:
        if num_positions > 0:
            close_position('all')
        print('Market for the symbol is currently closed')

        time.sleep(2)

    elif check_allowed_trading_hours() == True:
        # get the latest bar information
        rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, 1)
        price_data = rates[0]
        open = price_data[1]
        high = price_data[2]
        low = price_data[3]
        close = price_data[4]
        time_trade = datetime.fromtimestamp(price_data[0])
        print("open, high, low, close", price_data[1],price_data[2],price_data[3],price_data[4])
        print("Time:", time_trade)

        if time_trade not in time_records:
            url = "http://127.0.0.1:5000/predict_api"

            data = {
                "data": {
                    "open": open,
                    "high": high,
                    "low": low,
                    "close": close
                }
            }
            response = requests.post(url, json=data)

            if response.status_code == 200:
                prediction = response.json()
                print("make a trade at: ", time_trade)
                
            else:
                print("POST request failed!")
                print(response.status_code)

            if prediction == 1:
                order_result = market_order(symbol, volume, 'buy')
                time_records.append(time_trade)
                time_records.to_csv('time_records.csv')

            time.sleep(2)
    else:
        raise ValueError('Failed on Checking market status')
